In [ ]:
import sys

sys.path.append("../..")
import torch
from flashrnn.flashrnn import flashrnn
import logging

logging.basicConfig(level="INFO")

In [ ]:
device = "cuda"
dtype = torch.float32
B, T, G, H, D = 8, 2, 4, 1, 32
Wx = 1.0 + 0.0 * torch.randn([B, T, G, H, D], device=device, dtype=dtype)
R = 1 + 0.0 * torch.randn([G, H, D, D], device=device, dtype=dtype)
b = 1 + 0.0 * torch.randn([G, H, D], device=device, dtype=dtype)


Wx0 = Wx.detach().clone()
Wx0.requires_grad_(True)
R0 = R.detach().clone()
R0.requires_grad_(True)
b0 = b.detach().clone()
b0.requires_grad_(True)

h = flashrnn(Wx0, R0, b0, backend="cuda_fused", dtype="float32", function="slstm")
print(h[0].shape, h[1].shape)

h[0][0].sum().backward()

In [ ]:
Wx1 = Wx.detach().clone()
Wx1.requires_grad_(True)
R1 = R.detach().clone()
R1.requires_grad_(True)
b1 = b.detach().clone()
b1.requires_grad_(True)

h2 = flashrnn(Wx1, R1, b1, backend="vanilla", dtype="float32", function="slstm")
h2[0][0].sum().backward()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow((h[0] - h2[0]).reshape(64, 32).detach().cpu().abs(), vmin=0.0, vmax=0.0001)
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow((Wx0.grad - Wx1.grad).reshape(64, 32).detach().cpu().abs(), vmax=0.1)
plt.colorbar()

In [ ]:
Wx, R, b

In [ ]:
h[0], h2[0]

In [ ]:
b0.grad, b1.grad

In [ ]:
Wx1.grad - Wx0.grad

In [ ]:
Wx1.grad.shape, Wx1.grad

In [ ]:
import matplotlib.pyplot as plt

plt.imshow((b0.grad - b1.grad).reshape(4, 32).detach().cpu().abs(), vmax=0.1)
plt.colorbar()